In [1]:
from csbdeep.models import Config, CARE
import numpy as np
from csbdeep.utils import plot_some, plot_history
from csbdeep.utils.n2v_utils import manipulate_val_data

from matplotlib import pyplot as plt

import urllib

import os
import zipfile
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


In [2]:
# We need to normalize the data before we feed it into our network, and denormalize it afterwards.
def normalize(img, mean, std):
    zero_mean = img - mean
    return zero_mean/std

def denormalize(x, mean, std):
    return x*std + mean

In [3]:
model = CARE(None, 'n2v_model', basedir='/home/prakash/Desktop/falcon/outdata/n2v_CTC_n0/train_100/')

Loading network weights from 'weights_best.h5'.


In [4]:
path_train = '/home/prakash/Desktop/falcon/train_data/TrainVal.npz'
train_val = np.load(path_train)
X_train = train_val['X_train']
X_val = train_val['X_val']
Y_train = train_val['Y_train']
Y_val = train_val['Y_val']

In [5]:
path_train_N2V = '/home/prakash/Desktop/falcon/train_data/TrainValN2V.npz'
train_val_N2V = np.load(path_train_N2V)
X_train_N2V = train_val_N2V['X_train']
mean, std = np.mean(X_train_N2V), np.std(X_train_N2V)

In [6]:
path_test = '/home/prakash/Desktop/falcon/test_data/Test.npz'
test = np.load(path_test)
X_test = test['X_test']
Y_test = test['Y_test']

In [7]:
X_train = normalize(X_train, mean, std)
X_val = normalize(X_val, mean, std)
X_test = normalize(X_test, mean, std)

In [8]:
predictions = []
# Denoise all images
for i in range(X_train.shape[0]):
    predictions.append(denormalize(model.predict(X_train[i], axes='YX',normalizer=None ), mean, std))
X_train_d = np.array(predictions)

In [9]:
predictions = []
# Denoise all images
for i in range(X_val.shape[0]):
    predictions.append(denormalize(model.predict(X_val[i], axes='YX',normalizer=None ), mean, std))
X_val_d = np.array(predictions)

In [10]:
predictions = []
# Denoise all images
for i in range(X_test.shape[0]):
    predictions.append(denormalize(model.predict(X_test[i], axes='YX',normalizer=None ), mean, std))
X_test_d = np.array(predictions)

In [11]:
np.savez_compressed("/home/prakash/Desktop/falcon/train_data/TrainVal_denoised.npz", X_train=X_train_d, X_val=X_val_d,Y_train=Y_train, Y_val=Y_val)

In [12]:
np.savez_compressed("/home/prakash/Desktop/falcon/test_data/Test_denoised.npz", X_test=X_test_d, Y_test=Y_test)